# Boston Housing Price Prediction with scikit-learn (save model explanations via AML Run History)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

Explain a model with the AML explain-model package

1. Train a GradientBoosting regression model using Scikit-learn
2. Run 'explain_model' with AML Run History, which leverages run history service to store and manage the explanation data

# Save Model Explanation With AML Run History

In [ ]:
#Import Iris dataset
from sklearn import datasets
from sklearn.ensemble import GradientBoostingRegressor

import azureml.core
from azureml.core import Workspace, Experiment, Run
from azureml.explain.model.tabular_explainer import TabularExplainer
from azureml.contrib.explain.model.explanation.explanation_client import ExplanationClient
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
experiment_name = 'explain_model'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()
client = ExplanationClient.from_run(run)

## Load the Boston house price data

In [ ]:
boston_data = datasets.load_boston()

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(boston_data.data, boston_data.target, test_size=0.2, random_state=0)

## Train a GradientBoosting Regression model, which you want to explain

In [ ]:
clf = GradientBoostingRegressor(n_estimators=100, max_depth=4,
                                learning_rate=0.1, loss='huber',
                                random_state=1)
model = clf.fit(x_train, y_train)

## Explain predictions on your local machine

In [ ]:
tabular_explainer = TabularExplainer(model, x_train, features=boston_data.feature_names)

## Explain overall model predictions (global explanation)

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = tabular_explainer.explain_global(x_test)

In [ ]:
# Uploading model explanation data for storage or visualization in webUX
# The explanation can then be downloaded on any compute
client.upload_model_explanation(global_explanation)

In [ ]:
# Get model explanation data
explanation = client.download_model_explanation()
local_importance_values = explanation.local_importance_values
expected_values = explanation.expected_values

In [ ]:
# Print the values
print('expected values: {}'.format(expected_values))
print('local importance values: {}'.format(local_importance_values))

In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
explanation = client.download_model_explanation(top_k=4)
global_importance_values = explanation.get_ranked_global_values()
global_importance_names = explanation.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

## Explain individual instance predictions (local explanation) ##### needs to get updated with the new build

In [ ]:
local_explanation = tabular_explainer.explain_local(x_test[0,:])

In [ ]:
# local feature importance information
local_importance_values = local_explanation.local_importance_values
print('local importance values: {}'.format(local_importance_values))